# TD2 : FFT d'un sinus, effet des paramètres $N$ et $K$

Analyse du module de la FFT d'un sinus de fréquence fixe F0 (choisie volontairement comme un sous-multiple entier de $F_e$) observé à travers une fenêtre rectangulaire $w[n]$ de $N$ points.

L'analyse en fréquence peut être modifiée par 
- $N$ : règle le nombre d'échantillons (max 100) du signal sinusoidal,
- $K$ : nombre de points en fréquence de la FFT (zéro-padding), sélectionnable entre N et 5N.

Le signal temporel est affiché (graphique du haut) sur une durée fixe de 120 échantillons, complété éventuellement par
des zéros si $N<120$ pour la visualisation.

Le graphique central représente le tracé discret de la TFD (ou FFT) pour des fréquences $f$ comprises entre $0$ et $F_e$ dans 2 situations : sur $N$ points (en bleu) ou sur $K$ points (en rouge).

Le tracé inférieur zoome sur les deux FFT autour de 25 Hz avec des graduations plus fines.


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import IntSlider, interactive, HBox, VBox, Layout
from IPython.display import display
%matplotlib inline

FE = 200.0  # fréquence d'échantillonnage (Hz)
F0 = FE / 8  # F0 fixé comme sous-multiple de Fe
T_DISPLAY = 120 / FE  # durée fixe d'affichage (120 échantillons)
N_DISPLAY_SAMPLES = 120
ZOOM_FREQ = 25.0  # Hz
ZOOM_BAND = 5.0   # +/- autour de la fréquence de zoom
ZOOM_TICK = 0.5   # pas d'axe fréquence sur le zoom
AMPLITUDE = 1.0

def plot_fft_sinus(N, K):
    # Contraintes : N dans [2, 100], K suit N et reste dans [N, 5N]
    N = int(max(2, min(N, 100)))
    K = int(max(N, min(K, 5 * N)))

    # Signal sur N échantillons
    t_n = np.arange(N) / FE
    x_n = AMPLITUDE * np.sin(2 * np.pi * F0 * t_n)

    # Fenêtre d'affichage temporel : 120 échantillons, complétés par des zéros
    t_disp = np.arange(N_DISPLAY_SAMPLES) / FE
    x_disp = np.zeros(N_DISPLAY_SAMPLES)
    fill_len = min(N, N_DISPLAY_SAMPLES)
    x_disp[:fill_len] = x_n[:fill_len]

    # Zéro-padding pour la FFT K
    x_k = np.zeros(K)
    x_k[:N] = x_n

    # FFT sur N et K points
    X_N = np.fft.fft(x_n, n=N)
    X_K = np.fft.fft(x_k, n=K)
    freqs_N = np.arange(N) * FE / N
    freqs_K = np.arange(K) * FE / K

    mag_N = np.abs(X_N) / N
    mag_K = np.abs(X_K) / N

    fig, (ax_time, ax_freq, ax_zoom) = plt.subplots(3, 1, figsize=(11, 11))

    ax_time.set_title(
        f"Signal sinusoïdal sur 120 échantillons : F0 = {F0:.1f} Hz, Fe = {FE:.1f} Hz"
    )
    ax_time.stem(t_disp, x_disp, basefmt=" ", linefmt="b-", markerfmt="bo", label=f"x[n], N = {N}")
    ax_time.set_xlim(0, T_DISPLAY)
    ax_time.set_xlabel("Temps (s)")
    ax_time.set_ylabel("Amplitude")
    ax_time.grid(True)
    ax_time.legend(loc="upper right")

    ax_freq.set_title("Module de la FFT")
    ax_freq.stem(freqs_K, mag_K, basefmt=" ", linefmt="r-", markerfmt="r^", label=f"FFT K = {K}")
    ax_freq.stem(freqs_N, mag_N, basefmt=" ", linefmt="b-", markerfmt="bo", label=f"FFT N = {N}")
    ax_freq.set_xlim(0, FE)
    ax_freq.set_xlabel("Fréquence (Hz)")
    ax_freq.set_ylabel("|X(f)| normalisée")
    ax_freq.grid(True)
    ax_freq.legend(loc="upper right")

    # Zoom fréquentiel autour de 25 Hz
    f_min = max(0.0, ZOOM_FREQ - ZOOM_BAND)
    f_max = min(FE, ZOOM_FREQ + ZOOM_BAND)
    mask_N = (freqs_N >= f_min) & (freqs_N <= f_max)
    mask_K = (freqs_K >= f_min) & (freqs_K <= f_max)

    ax_zoom.set_title(f"Zoom FFT autour de {ZOOM_FREQ:.1f} Hz")
    if np.any(mask_K):
        ax_zoom.stem(freqs_K[mask_K], mag_K[mask_K], basefmt=" ", linefmt="r-", markerfmt="r^", label=f"FFT K = {K}")
    if np.any(mask_N):
        ax_zoom.stem(freqs_N[mask_N], mag_N[mask_N], basefmt=" ", linefmt="b-", markerfmt="bo", label=f"FFT N = {N}")
    ax_zoom.set_xlim(f_min, f_max)
    ax_zoom.set_xticks(np.arange(f_min, f_max + ZOOM_TICK / 2, ZOOM_TICK))
    ax_zoom.set_xlabel("Fréquence (Hz)")
    ax_zoom.set_ylabel("|X(f)| normalisée")
    ax_zoom.grid(True)
    ax_zoom.legend(loc="upper right")

    plt.tight_layout()
    plt.show()

slider_layout = Layout(width='420px')

N_slider = IntSlider(
    min=16, max=100, step=1, value=80,
    description='N (points) :',
    continuous_update=False,
    layout=slider_layout
)

K_slider = IntSlider(
    min=N_slider.value, max=5 * N_slider.value, step=1, value=N_slider.value,
    description='K (FFT) :',
    continuous_update=False,
    layout=slider_layout
)

def sync_K_with_N(change):
    N_val = N_slider.value
    K_slider.min = N_val
    K_slider.max = 5 * N_val
    K_slider.value = N_val

def clamp_K(change):
    N_val = N_slider.value
    if K_slider.value < N_val:
        K_slider.value = N_val
    elif K_slider.value > 5 * N_val:
        K_slider.value = 5 * N_val

N_slider.observe(sync_K_with_N, names='value')
K_slider.observe(clamp_K, names='value')

sync_K_with_N(None)

controls = HBox([N_slider, K_slider], layout=Layout(width='100%', justify_content='space-around'))
interactive_plot = interactive(plot_fft_sinus, N=N_slider, K=K_slider)

display(VBox([controls, interactive_plot.children[-1]]))
